# Homework 1: Finding Similar Items: Textually Similar Documents

In [1]:
import pandas as pd
import binascii
import timeit
from math import sqrt; from itertools import count, islice

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
#loads emails
dataset = pd.read_csv("filtered.csv")

In [4]:
#shows first 5 emails
dataset.head(5)

1
0  ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/09/2000 02:16 PM ---------------------------Richard Burchfield10/06/2000 06:59 AMTo: Phillip K Allen/HOU/ECT@ECTcc: Beth Perlman/HOU/ECT@ECT Subject: Consolidated positions: Issues & To Do listPhillip, Below is the issues & to do list as we go forward with documenting the requirements for consolidated physical/financial positions and transport trade capture. What we need to focus on is the first bullet in Allan's list; the need for a single set of requirements. Although the meeting with Keith, on Wednesday,  was informative the solution of creating a infinitely dynamic consolidated position screen, will be extremely difficult and time consuming.  Throughout the meeting on Wednesday, Keith alluded to the inability to get consensus amongst the traders on the presentation of the consolidated position, so the solution was to make it so that a trader can arrange the position screen to their liking (much like Excel). What needs to happen on Monday from 3 - 5 is a effort to design a desired layout for the consolidated position screen, this is critical. This does not exclude building a capability to create a more flexible position presentation for the future, but in order to create a plan that can be measured we need firm requirements. Also, to reiterate that the goals of this project is a project plan on consolidate physical/financial positions and transport trade capture. The other issues that have been raised will be capture as projects on to themselves, and will need to be prioritised as efforts outside of this project.I have been involved in most of the meetings and the discussions have been good. I believe there has been good communication between the teams, but now we need to have focus on the objectives we set out to solve.Richard  ---------------------- Forwarded by Richard Burchfield/HOU/ECT on 10/06/2000 08:34 AM ---------------------------Allan Severude10/05/2000 06:03 PMTo: Richard Burchfield/HOU/ECT@ECTcc: Peggy Alix/HOU/ECT@ECT, Russ Severson/HOU/ECT@ECT, Scott Mills/HOU/ECT@ECT, Kenny Ha/HOU/ECT@ECT Subject: Consolidated positions: Issues & To Do listFrom our initial set of meetings with the traders regarding consolidated positions, I think we still have the following issues:We don't have a single point of contact from the trading group.  We've had three meetings which brought out very different issues from different traders.  We really need a single point of contact to help drive the trader requirements and help come to a consensus regarding the requirements.We're getting hit with a lot of different requests, many of which appear to be outside the scope of position consolidation.Things left to do:I think it may be useful to try to formulate a high level project goal to make it as clear as possible what we're trying to accomplish with this project.  It'll help determine which requests fall under the project scope.Go through the list of requests to determine which are in scope for this project and which fall out of scope.For those in scope, work to define relative importance (priority) of each and work with traders to define the exact requirements of each.Define the desired lay out of the position manager screen: main view and all drill downs.Use the above to formulate a project plan.Things requested thus far (no particular order):Inclusion of Sitara physical deals into the TDS position manager and deal ticker.Customized rows and columns in the position manager (ad hoc rows/columns that add up existing position manager rows/columns).New drill down in the position manager to break out positions by: physical, transport, swaps, options, ...Addition of a curve tab to the position manager to show the real-time values of all curves on which the desk has a position.Ability to split the current position grid to allow daily positions to be shown directly above monthly positions.  Each grouped column in the top grid would be tied to a grouped column in the bot

## Shingles Similarity

In [5]:
# number of letters in a shingle
k_shingles=5

all_shingles=list() 
threshold=0.5

In [6]:
#function to compute the set of k-shingles for an email
def get_k_shingles(email_text):
    letters = list(str(email_text).strip('[]" ').replace("'",""))
    #for i in range(len(letters) - k_shingles + 1):
        #print (letters[i:i+k_shingles])
    #print(letters)
    return sorted(set([binascii.crc32(str.encode("".join(letters[i:i+k_shingles]))) & 0xffffffff for i in range(len(letters) - k_shingles + 1)]))

#function to compare two sets of k-shingles using Jaccard Similarity
def shingles_similarity(a, b):
    intersection = len(list(set(a).intersection(b)))
    #print (intersection)
    union = (len(a) + len(b)) - intersection
    if union == 0:
        return 0.0
    return intersection / float(union)

In [7]:
start = timeit.default_timer()

#iterates over the emails in the dataset and creates the shingles set for each mail calling the get_k_shingles function. It appends every shingles set in all_shingles.
for row in dataset.iterrows():
    email_shingles=get_k_shingles(row[1].values)
    #print(email_shingles)
    all_shingles.append(email_shingles)
    #print(all_shingles)

#prints the similarities between shingles sets whose value is > threshold
print("\n", "+ SHINGLES +","\n")
for i in range(len(all_shingles)):
    for j in range(len(all_shingles)):
        if i < j:
            value = shingles_similarity(all_shingles[i], all_shingles[j])
            if value > threshold:
                #print (all_shingles[i])
                #print (all_shingles[j])
                print("(mail " + str(i) + ", mail " + str(j) + "): " + str(value))
  


 + SHINGLES + 

(mail 0, mail 1): 0.9963963963963964
(mail 0, mail 22): 1.0
(mail 0, mail 23): 0.9963963963963964
(mail 0, mail 67): 0.9963963963963964
(mail 0, mail 68): 1.0
(mail 0, mail 115): 1.0
(mail 0, mail 116): 0.9963963963963964
(mail 1, mail 22): 0.9963963963963964
(mail 1, mail 23): 1.0
(mail 1, mail 67): 1.0
(mail 1, mail 68): 0.9963963963963964
(mail 1, mail 115): 0.9963963963963964
(mail 1, mail 116): 1.0
(mail 2, mail 3): 0.9980963259090044
(mail 2, mail 19): 0.9956298688960669
(mail 2, mail 24): 1.0
(mail 2, mail 25): 0.9980963259090044
(mail 2, mail 41): 0.9956298688960669
(mail 2, mail 65): 0.9980963259090044
(mail 2, mail 66): 1.0
(mail 2, mail 69): 0.9956298688960669
(mail 2, mail 110): 0.9956298688960669
(mail 2, mail 118): 1.0
(mail 2, mail 119): 0.9980963259090044
(mail 3, mail 19): 0.9956298688960669
(mail 3, mail 24): 0.9980963259090044
(mail 3, mail 25): 1.0
(mail 3, mail 41): 0.9956298688960669
(mail 3, mail 65): 1.0
(mail 3, mail 66): 0.9980963259090044
(ma

(mail 128, mail 371): 1.0
(mail 129, mail 173): 1.0
(mail 129, mail 308): 1.0
(mail 129, mail 372): 1.0
(mail 130, mail 174): 1.0
(mail 130, mail 373): 1.0
(mail 131, mail 175): 1.0
(mail 131, mail 374): 1.0
(mail 132, mail 176): 1.0
(mail 132, mail 375): 1.0
(mail 133, mail 137): 1.0
(mail 133, mail 376): 1.0
(mail 134, mail 135): 0.9255656587016282
(mail 134, mail 138): 1.0
(mail 134, mail 139): 0.9255656587016282
(mail 134, mail 307): 0.9255656587016282
(mail 134, mail 377): 1.0
(mail 134, mail 378): 0.9255656587016282
(mail 135, mail 138): 0.9255656587016282
(mail 135, mail 139): 1.0
(mail 135, mail 307): 1.0
(mail 135, mail 377): 0.9255656587016282
(mail 135, mail 378): 1.0
(mail 136, mail 140): 1.0
(mail 136, mail 306): 1.0
(mail 136, mail 379): 1.0
(mail 137, mail 376): 1.0
(mail 138, mail 139): 0.9255656587016282
(mail 138, mail 307): 0.9255656587016282
(mail 138, mail 377): 1.0
(mail 138, mail 378): 0.9255656587016282
(mail 139, mail 307): 1.0
(mail 139, mail 377): 0.925565658

(mail 402, mail 422): 0.6861391396707381
(mail 402, mail 442): 0.6145665485014502
(mail 402, mail 450): 1.0
(mail 403, mail 449): 1.0
(mail 405, mail 409): 0.705938242280285
(mail 405, mail 412): 0.7146960268070848
(mail 405, mail 417): 0.7421628189550425
(mail 405, mail 435): 0.7432792443690966
(mail 405, mail 447): 0.7517747858017136
(mail 406, mail 410): 0.8574669187145557
(mail 406, mail 413): 0.8638783269961977
(mail 406, mail 416): 0.854119425547997
(mail 406, mail 424): 0.863169897377423
(mail 406, mail 432): 0.8649162861491628
(mail 406, mail 433): 0.8545523233849641
(mail 406, mail 434): 0.8642257818459191
(mail 406, mail 436): 0.8727064220183486
(mail 406, mail 448): 0.8676582761250954
(mail 408, mail 460): 0.9812753036437247
(mail 409, mail 412): 0.7434146341463415
(mail 409, mail 417): 0.7026898357533921
(mail 409, mail 435): 0.6982738235989596
(mail 409, mail 447): 0.7097701149425287
(mail 410, mail 413): 0.8697303456133688
(mail 410, mail 416): 0.8522556390977444
(mail 41

In [8]:
#shows the content of an email
dataset.iloc[493]

1    Dan Douglass summarized this.  This really puts CA and the ISO on notice that they cannot confiscate the power as they seem ready to do -- FERC reiterates that the generators can sell power wherever they want because the cap is a cap on ISO purchases.  ANd if the ISO want to set a sale price cap it has to file with FERC, wait 60 days and amend its contract---------------------- Forwarded by Susan J Mara/SFO/EES on 08/01/2000 01:01 AM ---------------------------"Daniel Douglass" <douglass@ArterHadden.com> on 07/31/2000 07:27:24 PMTo: <peter.bray@att.net>, <JBarthrop@electric.com>, <mnelson@electric.com>, <rschlanert@electric.com>, <Bruno_Gaillard@enron.com>, <kmagrude@enron.com>, <mpetroch@enron.com>, <susan_j_mara@enron.com>, <athomas@newenergy.com>, <bchen@newenergy.com>, <Jeff.Hanson@phaser.com>, <anchau@shellus.com>, <andrew.madden@utility.com>, <ben.reyes@utility.com>, <chris.king@utility.com>, <david.bayless@utility.com>cc:  Subject: FERC Order on Morgan Stanley Complaint Aga

In [9]:
#shows the content of another email
dataset.iloc[496]

1    Dan Douglass summarized this.  This really puts CA and the ISO on notice that they cannot confiscate the power as they seem ready to do -- FERC reiterates that the generators can sell power wherever they want because the cap is a cap on ISO purchases.  ANd if the ISO want to set a sale price cap it has to file with FERC, wait 60 days and amend its contract---------------------- Forwarded by Susan J Mara/SFO/EES on 08/01/2000 01:01 AM ---------------------------"Daniel Douglass" <douglass@ArterHadden.com> on 07/31/2000 07:27:24 PMTo: <peter.bray@att.net>, <JBarthrop@electric.com>, <mnelson@electric.com>, <rschlanert@electric.com>, <Bruno_Gaillard@enron.com>, <kmagrude@enron.com>, <mpetroch@enron.com>, <susan_j_mara@enron.com>, <athomas@newenergy.com>, <bchen@newenergy.com>, <Jeff.Hanson@phaser.com>, <anchau@shellus.com>, <andrew.madden@utility.com>, <ben.reyes@utility.com>, <chris.king@utility.com>, <david.bayless@utility.com>cc:  Subject: FERC Order on Morgan Stanley Complaint Aga

In [10]:
all_shingles_counter=0
for email_shingles in all_shingles:
    for shingle in email_shingles:
        all_shingles_counter=all_shingles_counter+1
print ("global n. of shingles: ",all_shingles_counter)
print ("avg n. of shingles per email: ", all_shingles_counter/500)

global n. of shingles:  3054354
avg n. of shingles per email:  6108.708


## MinHash Similarity

In [11]:
#the max integer as the upper limit of the minhashes
max_integer=2**32-1

In [12]:
#checks if integer n is a prime
def isPrime(n):
    return n > 1 and all(n%i for i in islice(count(2), int(sqrt(n)-1)))

In [13]:
i = 1
#finds first prime which is higher than the max integer
while not isPrime(max_integer + i):
    i = i + 1
c = max_integer+i
c

4294967311

In [14]:
import random
import numpy as np

#number of hashes in the MinHash step
num_hashes=100
#matrix of signatures
signature_matrix = [0 for i in range(num_hashes)]

In [15]:
#function to get as many pairs of parameters as the number of hash functions
def get_hash_coefficients():
    a = random.sample(range(0,max_integer),num_hashes)
    b = random.sample(range(0,max_integer),num_hashes)
    return a, b

#function to compute the signature out of a set of shingles
def signature(email_shingles,a,b):  
    #the resulting minhash signature for this email
    signature = []
    #for each of the random hash functions...
    for i in range(num_hashes):
        #tracks the lowest hash ID seen
        #initializes 'min_hash' to be greater than the maximum possible value output by the hash
        min_hash = c + 1
        for shingle in email_shingles:
            #print("shingle: ",shingle)
            #our random hash function will take the form of:
            #h(x) = (a*x + b) % c
            #where 'x' is the input value, 'a' and 'b' are random coefficients, and 'c' is
            #a prime number just greater than shingleNo.
            hash_code = (a[i] * shingle + b[i]) % c 
            #print ("hash: ",hash_code)
            if hash_code < min_hash:
                min_hash = hash_code
        signature.append(min_hash)
    #print (signature)
    return signature

#function to compare two signatures
def compare_signatures( a, b):
    #print ("a: ", a)
    #print ("b: ", b)
    equal = 0
    for i in range(len(a)):
        if(a[i]==b[i]):
            equal += 1
    return equal/len(a)

In [16]:
a,b = get_hash_coefficients()
#iterates over the emails in the dataset and creates the signature for each mail calling the signature function. Meanwhile, it appends every signature in the signature_matrix.
for row in dataset.iterrows():
    #gets the shingles set for this email
    email_shingles=get_k_shingles(row[1].values)
    #print (signature(email_shingles, a, b))
    signature_matrix = np.column_stack((signature_matrix, signature(email_shingles, a, b)))

In [17]:
signature_matrix = np.delete(signature_matrix,0,axis = 1)
signature_matrix.shape[1]

500

In [18]:
#prints the similarities between signatures whose value is > threshold
print("\n", "+ MIN-HASH +","\n")
k=0
for i in range(signature_matrix.shape[1]):
    for j in range(signature_matrix.shape[1]):
        if i < j:
            #print(value)
            value = compare_signatures(np.squeeze(signature_matrix[:,i]), np.squeeze(signature_matrix[:,j]))
            if value > threshold:
                k=k+1
                print("(mail " + str(i) + ", mail " + str(j) + "): " + str(value))


 + MIN-HASH + 

(mail 0, mail 1): 0.99
(mail 0, mail 22): 1.0
(mail 0, mail 23): 0.99
(mail 0, mail 67): 0.99
(mail 0, mail 68): 1.0
(mail 0, mail 115): 1.0
(mail 0, mail 116): 0.99
(mail 1, mail 22): 0.99
(mail 1, mail 23): 1.0
(mail 1, mail 67): 1.0
(mail 1, mail 68): 0.99
(mail 1, mail 115): 0.99
(mail 1, mail 116): 1.0
(mail 2, mail 3): 1.0
(mail 2, mail 19): 0.99
(mail 2, mail 24): 1.0
(mail 2, mail 25): 1.0
(mail 2, mail 41): 0.99
(mail 2, mail 65): 1.0
(mail 2, mail 66): 1.0
(mail 2, mail 69): 0.99
(mail 2, mail 110): 0.99
(mail 2, mail 118): 1.0
(mail 2, mail 119): 1.0
(mail 3, mail 19): 0.99
(mail 3, mail 24): 1.0
(mail 3, mail 25): 1.0
(mail 3, mail 41): 0.99
(mail 3, mail 65): 1.0
(mail 3, mail 66): 1.0
(mail 3, mail 69): 0.99
(mail 3, mail 110): 0.99
(mail 3, mail 118): 1.0
(mail 3, mail 119): 1.0
(mail 4, mail 10): 0.86
(mail 4, mail 32): 0.86
(mail 4, mail 81): 0.86
(mail 4, mail 102): 1.0
(mail 4, mail 107): 0.86
(mail 5, mail 26): 1.0
(mail 5, mail 64): 1.0
(mail 5, ma

(mail 161, mail 187): 1.0
(mail 162, mail 314): 1.0
(mail 162, mail 353): 0.97
(mail 162, mail 365): 1.0
(mail 163, mail 312): 1.0
(mail 163, mail 313): 0.89
(mail 163, mail 339): 1.0
(mail 163, mail 347): 0.86
(mail 163, mail 362): 0.89
(mail 164, mail 167): 0.87
(mail 164, mail 181): 1.0
(mail 164, mail 182): 0.87
(mail 165, mail 186): 1.0
(mail 166, mail 170): 0.73
(mail 166, mail 311): 1.0
(mail 166, mail 366): 1.0
(mail 166, mail 369): 0.73
(mail 167, mail 181): 0.87
(mail 167, mail 182): 1.0
(mail 168, mail 310): 1.0
(mail 168, mail 367): 1.0
(mail 169, mail 171): 0.56
(mail 169, mail 368): 1.0
(mail 169, mail 370): 0.56
(mail 170, mail 311): 0.73
(mail 170, mail 366): 0.73
(mail 170, mail 369): 1.0
(mail 171, mail 368): 0.56
(mail 171, mail 370): 1.0
(mail 172, mail 309): 1.0
(mail 172, mail 371): 1.0
(mail 173, mail 308): 1.0
(mail 173, mail 372): 1.0
(mail 174, mail 373): 1.0
(mail 175, mail 374): 1.0
(mail 176, mail 375): 1.0
(mail 177, mail 179): 0.67
(mail 181, mail 182): 0

In [19]:
#shows the content of an email
dataset.iloc[4]

1    ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/01/2001 0=2:14 PM ---------------------------Outlook Migration Team@ENRON04/27/2001 01:01 PMTo:=09Allison Horton/NA/Enron@ENRON, Amir Baig/NA/Enron@ENRON, Brandon Bang=erter/NA/Enron@Enron, Brian Ellis/Corp/Enron@Enron, Charles Philpott/HR/Cor=p/Enron@ENRON, Chris P Wood/NA/Enron@Enron, Chris Tull/HOU/ECT@ECT, Dale Sm=ith/Corp/Enron@ENRON, Dave June/NA/Enron@ENRON, Donald Sutton/NA/Enron@Enro=n, Felicia Buenrostro/HR/Corp/Enron@ENRON, Johnna Morrison/Corp/Enron@ENRON=, Joe Dorn/Corp/Enron@ENRON, Kathryn Schultea/HR/Corp/Enron@ENRON, Leon McD=owell/NA/Enron@ENRON, Leticia Barrios/Corp/Enron@ENRON, Milton Brown/HR/Cor=p/Enron@ENRON, Raj Perubhatla/Corp/Enron@Enron, Shekar Komatireddy/NA/Enron=@Enron, Andrea Yowman/Corp/Enron@ENRON, Angie O'Brian/HR/Corp/Enron@ENRON, =Bonne Castellano/HR/Corp/Enron@ENRON, Gwynn Gorsuch/NA/Enron@ENRON, Jo Ann =Matson/Corp/Enron@ENRON, LaQuitta Washington/HR/Corp/Enron@ENRON, Rick John=s

In [20]:
#shows the content of another email
dataset.iloc[10]

1    ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/01/2001 02:14 PM ---------------------------Outlook Migration Team@ENRON04/27/2001 01:01 PMTo: Allison Horton/NA/Enron@ENRON, Amir Baig/NA/Enron@ENRON, Brandon Bangerter/NA/Enron@Enron, Brian Ellis/Corp/Enron@Enron, Charles Philpott/HR/Corp/Enron@ENRON, Chris P Wood/NA/Enron@Enron, Chris Tull/HOU/ECT@ECT, Dale Smith/Corp/Enron@ENRON, Dave June/NA/Enron@ENRON, Donald Sutton/NA/Enron@Enron, Felicia Buenrostro/HR/Corp/Enron@ENRON, Johnna Morrison/Corp/Enron@ENRON, Joe Dorn/Corp/Enron@ENRON, Kathryn Schultea/HR/Corp/Enron@ENRON, Leon McDowell/NA/Enron@ENRON, Leticia Barrios/Corp/Enron@ENRON, Milton Brown/HR/Corp/Enron@ENRON, Raj Perubhatla/Corp/Enron@Enron, Shekar Komatireddy/NA/Enron@Enron, Andrea Yowman/Corp/Enron@ENRON, Angie O'Brian/HR/Corp/Enron@ENRON, Bonne Castellano/HR/Corp/Enron@ENRON, Gwynn Gorsuch/NA/Enron@ENRON, Jo Ann Matson/Corp/Enron@ENRON, LaQuitta Washington/HR/Corp/Enron@ENRON, Rick Johnson/HR/Corp/Enr

## LSH Similarity

In [21]:
from operator import itemgetter
from itertools import combinations

len_buckets = 1000000

#initializes the buckets
def initialize_array_bucket(bands):
    global len_buckets
    array_buckets = []
    for band in range(bands):
        array_buckets.append([[] for i in range(len_buckets)])
    return array_buckets

def euclidean_distance(x,y,r=2.0):
    return sum(((x[i] - y[i]) ** r) for i in range(len(x))) ** (1.0/r)

def cosine_distance(x,y):
    prodAB = sum([x[i]*y[i] for i in range(len(x))])
    zeros = [0 for i in range(len(x))]
    A = euclidean_distance(x,zeros)
    B = euclidean_distance(y,zeros)
    return prodAB / (A*B)

#function that applies LSH technique
def apply_LSH_technique(signature_matrix, threshold, bands, rows):
    #checks that number of bands b and number of rows r such that br = n
    if bands * rows != len(signature_matrix):
        raise 'bands*rows must be equals to n :: bands*rows = n !!!'
    
    array_buckets = initialize_array_bucket(bands)

    #initializes number of hashes as the number of bands
    num_hashes=bands
    
    #initializes candidate pairs
    candidates = {}
    
    #for each band: we compute the hash function for each column in it and assign that column to the related bucket
    i = 0
    m,n= get_hash_coefficients()
    for b in range(bands):
        buckets = array_buckets[b]        
        band = signature_matrix[i:i+rows,:]
        for col in range(band.shape[1]):
            key = (m[i] * sum(band[:,col]) + n[i]) % len(buckets)
            buckets[key].append(col)
        i = i+rows
        
        #for each bucket in each band we check the possible emails pairs and evaluate their similarity
        for bucket in buckets:
            if len(bucket) > 1:
                candidates_bucket = combinations(bucket, 2)
                for pair in candidates_bucket:
                    if pair not in candidates:
                        a = signature_matrix[:,pair[0]]
                        b = signature_matrix[:,pair[1]]
                        similarity = cosine_distance(a,b)
                        if similarity >= threshold:
                            candidates[pair] = similarity
    return candidates

In [22]:
#threshold is approximately (1/b)^1/r
t=0.55
b=20
r=5

#calls the LSH function
candidates = apply_LSH_technique(signature_matrix,t,b,r)
stop = timeit.default_timer()

print("\n", "+ LSH +","\n")
print("There are " + str(len(candidates)) + " candidate pairs\n")

print(candidates)

print ("Time: ",stop-start)


 + LSH + 

There are 729 candidate pairs

{(6, 103): 1.0, (144, 322): 1.0000000000000002, (165, 186): 0.9999999999999998, (381, 439): 1.0, (381, 457): 1.0, (439, 457): 1.0, (123, 126): 0.8475967341050207, (123, 166): 1.0, (123, 170): 0.8475967341050207, (123, 311): 1.0, (123, 366): 1.0, (123, 369): 0.8475967341050207, (126, 166): 0.8475967341050207, (126, 170): 1.0000000000000002, (126, 311): 0.8475967341050207, (126, 366): 0.8475967341050207, (126, 369): 1.0000000000000002, (166, 170): 0.8475967341050207, (166, 311): 1.0, (166, 366): 1.0, (166, 369): 0.8475967341050207, (170, 311): 0.8475967341050207, (170, 366): 0.8475967341050207, (170, 369): 1.0000000000000002, (311, 366): 1.0, (311, 369): 0.8475967341050207, (366, 369): 0.8475967341050207, (9, 31): 1.0000000000000002, (9, 82): 1.0000000000000002, (9, 106): 1.0000000000000002, (31, 82): 1.0000000000000002, (31, 106): 1.0000000000000002, (82, 106): 1.0000000000000002, (160, 315): 1.0000000000000002, (160, 338): 1.0000000000000002, 

In [23]:
#shows the content of an email
dataset.iloc[157]

1    Hawk vote for California firm unanimous Houston Chronicle, 05/15/01INTERNATIONAL ECONOMY: Enron may cut stake in Gulf gas project Financial Times; May 15, 2001JAPAN: Enron says high power rates costing Japan.Reuters English News Service, 05/15/01Japan Must Speed Up Pwr Sector Dereg To Lower Rates-IndusDow Jones Energy Service, 05/15/01SINGAPORE: ANALYSIS-No Asia fallout seen from Enron's India woes.Reuters English News Service, 05/15/01Saudi Won't Announce Winners Of Gas Projs Tue - ReportDow Jones Energy Service, 05/15/01MSEB refutes allegations by Enron, DPCThe Economic Times, 05/15/01Saudi Supreme Petrol Council meeting to decide on huge gas project bidsBusiness Recorder, 05/15/01May 15, 2001Houston ChronicleHawk vote for California firm unanimous Montgomery Watson pegged for water plant By MARY FLOOD Copyright 2001 Houston Chronicle The Houston Area Water Corp. voted unanimously Monday to grant a $92 million contract to a California-based firm to design, build and operate a La

In [24]:
#shows the content of another email
dataset.iloc[317]

1    Hawk vote for California firm unanimous Houston Chronicle, 05/15/01INTERNATIONAL ECONOMY: Enron may cut stake in Gulf gas project Financial Times; May 15, 2001JAPAN: Enron says high power rates costing Japan.Reuters English News Service, 05/15/01Japan Must Speed Up Pwr Sector Dereg To Lower Rates-IndusDow Jones Energy Service, 05/15/01SINGAPORE: ANALYSIS-No Asia fallout seen from Enron's India woes.Reuters English News Service, 05/15/01Saudi Won't Announce Winners Of Gas Projs Tue - ReportDow Jones Energy Service, 05/15/01MSEB refutes allegations by Enron, DPCThe Economic Times, 05/15/01Saudi Supreme Petrol Council meeting to decide on huge gas project bidsBusiness Recorder, 05/15/01May 15, 2001Houston ChronicleHawk vote for California firm unanimous Montgomery Watson pegged for water plant By MARY FLOOD Copyright 2001 Houston Chronicle The Houston Area Water Corp. voted unanimously Monday to grant a $92 million contract to a California-based firm to design, build and operate a La